In [1]:
import os
import pickle
from tqdm.notebook import tqdm
import sentencepiece as spm

In [2]:
# source_folder = '/scratche/home/apoorv/EmbedKGQA/data/MetaQA'
source_folder = 'data/fbwq_half_lego'
! ls {source_folder}

entity_strings.txt  qa_train_1hop.txt  train.txt
qa_test_1hop.txt    test.txt	       valid.txt


In [3]:
def readLines(fname):
    f = open(fname)
    lines = []
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        lines.append(line)
    f.close()
    return lines

In [4]:
data = {}
hops = ['1']
for hop in hops:
    fname = os.path.join(source_folder, 'qa_train_'+ hop + 'hop.txt')
    data[hop] = readLines(fname)

In [5]:
data['1'][2]

'what country is the [grand bahama] in\tbahamas'

In [6]:
from unidecode import unidecode
def normalizeEntity(ent):
    ent = unidecode(ent)
    return ent

In [7]:
def separateEntity(question, replacement='NE'):
    split1 = question.split('[')
    lhs = split1[0]
    split2 = split1[1].split(']')
    entity = split2[0]
    rhs = split2[1]
    final = lhs + replacement + rhs
    return final, entity

def lineToKGCLines(line):
    line = line.split('\t')
    question, head_entity = separateEntity(line[0])
    head_entity = normalizeEntity(head_entity)
    base_line = 'predict answer: {0} | {1} |'.format(head_entity, question)
    answers = line[1].split('|') # multiple entities can be answer
    answers = [normalizeEntity(a) for a in answers]
    kgc_lines = []
    for a in answers:
        kgc_lines.append(base_line + '\t' + a)
    return kgc_lines

In [8]:
lineToKGCLines(data['1'][0])

['predict answer: justin bieber | what is the name of NE brother |\tjaxon bieber']

In [9]:
kgc_lines_1hop = []
for line in data['1']:
    kgc_lines_1hop.extend(lineToKGCLines(line))

In [10]:
len(kgc_lines_1hop)

35358

In [11]:
def writeLines(lines, fname):
    f = open(fname, 'w')
    for line in lines:
        f.write(line + '\n')
    f.close()

In [12]:
fname = 'data/fbwq_half_lego_mixed/train_base.txt'
writeLines(kgc_lines_1hop, fname)